In [190]:
#IMPORT MODULES
#IMPORT MODULES
import pandas as pd
import numpy as np
import time
# SENTIMENT ANALYSIS USING VADER
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [191]:
#READ THE DATA FILE
df1 = pd.read_csv("../Datasets/survey_results_public.csv")

In [192]:
#ADD COLUMNS HERE

#Employment Column Additions
column_name = "Employment"
st = set()
for i in range (0,len(df1)):
    value = str(df1[column_name].iloc[i])
    if(value == "nan"):continue
    l = value.split(";")
    for ele in l:
        st.add(ele)
for ele in st:
    df1[ele] = 0
    
for i in range (0,len(df1)):
    value = str(df1[column_name].iloc[i])
    if(value == "nan"):continue
    l = value.split(";")
    for ele in l:
        df1.loc[i,ele] = 1

#Countries filtered out if not India
df1 = df1[df1["Industry"] == "Information Services, IT, Software Development, or other Technology"]
#Industries filtered out if not Information Services, IT, Software Development, or other Technology
df1 = df1[df1["Country"] == "India"]

#Number of Languages known
column_name = "LanguageHaveWorkedWith"
def languagecount(row):
    value = str(row[column_name]).split(";")
    if(value[0] == "nan"):return 0
    return len(value)
df1["NumberOfLanguagesKnown"] = df1.apply(languagecount,axis = 1)

# # #Number of Databases known
column_name = "DatabaseHaveWorkedWith"
def languagecount(row):
    value = str(row[column_name]).split(";")
    if(value[0] == "nan"):return 0
    return len(value)
df1["NumberOfDatabasesKnown"] = df1.apply(languagecount,axis = 1)

In [193]:
columns_selected =  [
   'AISelect',
   'OrgSize',
   'DevType',
   'YearsCode',
   'WorkExp', 
   'YearsCodePro',
   "RemoteWork",
   'Currency',
   "EdLevel",
   "ConvertedCompYearly",
   "NumberOfDatabasesKnown",
   "NumberOfLanguagesKnown"
]

train_columns = [
   'AISelect',
   'OrgSize',
   'DevType',
   "RemoteWork",
   'Currency',
   "EdLevel",
   "ExperienceCategory",
   "YearsCodeCategory",
   "YearsCodeProCategory",
   "NumberOfDatabasesKnown",
   "NumberOfLanguagesKnown"
]

for ele in st:
    train_columns.append(ele)
    columns_selected.append(ele)

df1 = df1[columns_selected]

df1 = df1.dropna()

In [194]:
#CATEGORISE COLUMNS  INTO MAJORITY VALUES AND 'OTHER'
def shorten_categories(categories, cutoff):
    categorical_map = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff:
            categorical_map[categories.index[i]] = categories.index[i]
        else:
            categorical_map[categories.index[i]] = 'Other'
    return categorical_map


currency_map = shorten_categories(df1.Currency.value_counts(), 400)
df1['Currency'] = df1['Currency'].map(currency_map)

In [195]:
#CATEGORISE THE WORK EXPERIENCE INTO BINS
bins = [0, 2, 5, 10, 20, 30, 40, 50, float('inf')]  # Define custom bin edges
labels = [0, 1, 2, 3, 4, 5, 6, 7]  # Define labels

# Create a new column with the categories
df1['ExperienceCategory'] = pd.cut(df1['WorkExp'], bins=bins, labels=labels)

In [196]:
#CATEGORISE LESS THAN 1 YEAR AS 0 AND MORE THAN 50 AS 51 FOR YEARS OF CODE
df1['YearsCode'] = df1['YearsCode'].replace("Less than 1 year", 0)
df1['YearsCode'] = df1['YearsCode'].replace("More than 50 years", 51)

df1['YearsCodePro'] = df1['YearsCodePro'].replace("Less than 1 year", 0)
df1['YearsCodePro'] = df1['YearsCodePro'].replace("More than 50 years", 51)

In [197]:
#CATEGORISE YEARS OF CODE INTO BINS
bins = [0, 2, 5, 10, 20, 30, 40, 50, float('inf')]  # Define custom bin edges
labels = [0, 1, 2, 3, 4, 5, 6, 7]  # Define labels

# Create a new column with the categories
df1["YearsCode"] = df1["YearsCode"].astype(int)
df1["YearsCodePro"] = df1["YearsCodePro"].astype(int)
df1['YearsCodeCategory'] = pd.cut(df1['YearsCode'], bins=bins, labels=labels)
df1['YearsCodeProCategory'] = pd.cut(df1['YearsCodePro'], bins=bins, labels=labels)

In [198]:
len(df1)

882

In [199]:
#LABEL ENCODE THE COLUMNS

df_LE = df1
df_LE = df_LE.dropna()

for i in train_columns:
    if i == "ConvertedCompYearly":
        continue
    le = LabelEncoder()
    df_LE[i] = le.fit_transform(df_LE[i])

X = df_LE[train_columns]

Y = df_LE["ConvertedCompYearly"]

C:\Users\Siddharth Shah\AppData\Local\Temp\ipykernel_16984\2931493265.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LE[i] = le.fit_transform(df_LE[i])
C:\Users\Siddharth Shah\AppData\Local\Temp\ipykernel_16984\2931493265.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LE[i] = le.fit_transform(df_LE[i])
C:\Users\Siddharth Shah\AppData\Local\Temp\ipykernel_16984\2931493265.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [200]:
#TRAIN-TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.1, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [206]:
classifiers = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=1.0),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "XGBoost" : XGBRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Support Vector Machine": SVR(),
    "K-Nearest Neighbors": KNeighborsRegressor(),
    "Neural Network": MLPRegressor(),
    "Gaussian Process": GaussianProcessRegressor(),
}

In [207]:
#CHECK IMPORTANCE
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X, Y)
feature_importances = model.feature_importances_

map_ = {}

for i, a in enumerate(X):
    map_[a] = feature_importances[i]
    
sorted(map_.items(), key = lambda x: x[1])

[('Retired', 0.0),
 ('Student, full-time', 0.0),
 ('Not employed, and not looking for work', 0.0),
 ('Not employed, but looking for work', 0.0),
 ('I prefer not to say', 0.0),
 ('Student, part-time', 0.0),
 ('Employed, part-time', 0.001532586876589646),
 ('Independent contractor, freelancer, or self-employed',
  0.008169586733499316),
 ('Employed, full-time', 0.015230476098910239),
 ('EdLevel', 0.03131184102314637),
 ('Currency', 0.031430956908519825),
 ('RemoteWork', 0.04172531022852646),
 ('YearsCodeProCategory', 0.05664253006112379),
 ('AISelect', 0.07717359821445231),
 ('YearsCodeCategory', 0.07899864742054462),
 ('NumberOfDatabasesKnown', 0.10418228050672855),
 ('OrgSize', 0.11319423468926543),
 ('DevType', 0.12713722540540148),
 ('ExperienceCategory', 0.1515461936498178),
 ('NumberOfLanguagesKnown', 0.1617245321834742)]

In [208]:
# Create an empty DataFrame to store the results
columns = ['Model', 'Run Time (minutes)', 'MAE', 'MSE', 'RMSE', 'R2']
df_models = pd.DataFrame(columns=columns)

# Loop through your regression models
for key, clf in classifiers.items():
    # STARTING TIME
    start_time = time.time()
    # TRAIN CLASSIFIER ON TRAINING DATA
    clf.fit(X_train_scaled, y_train)
    # MAKE PREDICTIONS USING CURRENT CLASSIFIER
    predictions = clf.predict(X_test_scaled)
    # CALCULATE REGRESSION METRICS
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = mean_squared_error(y_test, predictions, squared=False)  # Calculate RMSE
    r2 = r2_score(y_test, predictions)

    row = {'Model': key,
           'Run Time (minutes)': round((time.time() - start_time) / 60, 2),
           'MAE': mae,
           'MSE': mse,
           'RMSE': rmse,
           'R2': r2
           }

    df_models = pd.concat([df_models, pd.DataFrame([row])], ignore_index=True)

# Sort the DataFrame by R-squared (R2) in descending order
df_models = df_models.sort_values(by='R2', ascending=False)

# PRINT THE MODELS WITH REGRESSION METRICS [SORTED]
print(df_models)

C:\Users\Siddharth Shah\AppData\Local\Temp\ipykernel_16984\201078824.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_models = pd.concat([df_models, pd.DataFrame([row])], ignore_index=True)
c:\Users\Siddharth Shah\anaconda3\envs\eightfoldai\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


                     Model  Run Time (minutes)           MAE           MSE  \
1                    Ridge                0.00   9578.425327  1.635169e+08   
2                    Lasso                0.00   9577.609948  1.635375e+08   
0        Linear Regression                0.00   9578.125221  1.635578e+08   
4            Random Forest                0.01  10614.228402  1.909670e+08   
6        Gradient Boosting                0.00  11015.299810  2.207475e+08   
7   Support Vector Machine                0.00  12751.956845  2.677033e+08   
10        Gaussian Process                0.00  15087.124941  4.240044e+08   
5                  XGBoost                0.00  15671.693453  4.619197e+08   
8      K-Nearest Neighbors                0.00  14317.238554  4.908062e+08   
9           Neural Network                0.01  19895.898522  6.594669e+08   
3            Decision Tree                0.00  16612.355422  7.740811e+08   

            RMSE            R2  
1   12787.374547  3.891858e-01

c:\Users\Siddharth Shah\anaconda3\envs\eightfoldai\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
